In [1]:
from fast_to_SQL.fast_to_SQL import to_sql_fast
import pandas as pd
from urllib.parse import quote_plus
import sqlalchemy
%matplotlib inline

params = quote_plus(
    """DRIVER={ODBC Driver 13 for SQL Server};
    SERVER=server-dw01.database.windows.net;
    DATABASE=DataWarehouseStaging;
    UID=dw_userlogin;
    PWD=kljDueK!7"""
)

ENGINE = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [2]:
month_no_dict = {'jan' : 1,
'feb' : 2,
'mar' : 3,
'apr' : 4,
'may' : 5,
'jun' : 6,
'jul' : 7,
'aug' : 8,
'sep' : 9,
'oct' : 10,
'nov' : 11,
'dec' : 12
}

In [3]:
Xer_df = pd.read_sql(''' SELECT * FROM pam.dataset_Xer ''', ENGINE)

In [4]:
df = pd.melt(Xer_df, id_vars = ['com_id_h'], var_name = 'month', value_name = 'Xer')

In [5]:
df['month_no'] = df['month'].map(month_no_dict)

In [6]:
mktg_df = pd.read_sql(''' SELECT [Hashed_ID], [Land_Date], [Opportunity ID], [MCC_Description], t.[Tenure]
	,t.[Average_Turnover]
	,t.[Average_Trans_Count]
	,t.[Times_Marketed_To]
	,t.[PA_Marketing_Flag] AS [Message]
    ,t.[Pam_Score]
    ,t.[Underwriting Decision]
    ,t.[Fund Date]
                            FROM [marketing].[WP_Attri_Mailers] AS t
                            WHERE Marketed_To = 1''', ENGINE)

In [7]:
mktg_df['Land_Date'] = pd.to_datetime(mktg_df['Land_Date'])

mktg_df['land_month'] = pd.DatetimeIndex(mktg_df['Land_Date']).month

import calendar
mktg_df['land_month'] = mktg_df['land_month'].apply(lambda x: calendar.month_abbr[x])

mktg_df['land_month'] = mktg_df['land_month'].str.lower()

In [8]:
mktg_df['land_month_no'] = mktg_df['land_month'].map(month_no_dict)

In [9]:
mktg_df['responded'] = mktg_df['Opportunity ID'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [10]:
mktg_df['approved'] = mktg_df['Underwriting Decision'].apply(lambda x: 0 if x == 'Approve' else 1)

In [11]:
mktg_df['funded'] = mktg_df['Fund Date'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [12]:
final_df = mktg_df.merge(right = df, how = 'left', left_on = ['Hashed_ID','land_month_no'], right_on = ['com_id_h', 'month_no'])

In [13]:
final_df.columns

Index(['Hashed_ID', 'Land_Date', 'Opportunity ID', 'MCC_Description', 'Tenure',
       'Average_Turnover', 'Average_Trans_Count', 'Times_Marketed_To',
       'Message', 'Pam_Score', 'Underwriting Decision', 'Fund Date',
       'land_month', 'land_month_no', 'responded', 'approved', 'funded',
       'com_id_h', 'month', 'Xer', 'month_no'],
      dtype='object')

In [14]:
good_mcc = ['RESTAURANTS (EXCEPT EXPRESS)', 'EXPRESS PAYMNT SRVCE FAST FOOD', 'DRINKING PLACES (ALCOHOLIC)',
            'MISCELLANEOUS FOOD STORES', 'WOMENS READY TO WEAR STORES', 'GROCERY STORES SUPERMARKETS', ]

bad_mcc = ['AUTOMOTIVE SERVICE SHOPS', 'BUSINESS SERVICES MISC', 'AUTOMOTIVE PARTS ACCESSORIES', 'MEDICAL SERVICES MISC',
          'LUMBER & BUILDING MATERIALS', 'DENTISTS ORTHODONTISTS', 'PROFESSIONAL SERVICES', 'JEWELLERY STORES WATCHES CLOCK',
          'LEGAL SERVICES ATTORNEYS']

In [15]:
def mcc_rating_func(item):
    if item in good_mcc:
        return 1
    elif item in bad_mcc:
        return -1
    else:
        return 0
    
final_df['mcc_rating'] = final_df['MCC_Description'].apply(mcc_rating_func)

In [16]:
industry_dict = {'health/beauty':
                ['BARBER & BEAUTY SHOPS', 'HEALTH & BEAUTY SPAS'],
                 
                'food/drink/hotel':
                ['RESTAURANTS (EXCEPT EXPRESS)', 'EXPRESS PAYMNT SRVCE FAST FOOD', 'HOTELS MOTELS & RESORTS-LODGIN',
                'DRINKING PLACES (ALCOHOLIC)', 'MISCELLANEOUS FOOD STORES', 'GROCERY STORES SUPERMARKETS',
                'PACKAGE STORES-BEER WINE & LIQ', 'CATERERS'],
                 
                'medical/dental/vets':
                 ['MEDICAL SERVICES MISC', 'DENTISTS ORTHODONTISTS', 'OPTICIANS',
                 'DRUG STORES & PHARMACIES', 'VETERINARY SERVICES', 'CHIROPRACTORS', 'DOCTORS-NOT ELSEWHERE CLASS.',
                 'OSTEOPATHS', 'LAB/DENT/OPTH-HOSP EQPT/SUP(PC', 'CHIROPODISTS', 'NURSING & PERSONAL CARE FACILI',
                 'OPTOMETRISTS OPHTHALMOLOGISTS'],
                 
                'auto':
                 ['AUTOMOTIVE SERVICE SHOPS', 'AUTOMOTIVE PARTS ACCESSORIES', 'AUTOMOTIVE BODY REPAIR SHOPS',
                 'AUTOMOTIVE TYRE STORES', 'MOTORCYCLE SHOPS & DEALERS', 'BICYCLE SHOPS-SALES & SERVICE',
                 'AUTO RENTALS', 'FUEL DEALERS', 'MARINAS MARINE SERVICES & SUPP', 'MOTOR VEHICLE SUP/NEW PTS(P/C)',
                 'AUTO AIRCRAFT FARM EQP.', 'BOAT RENTALS & LEASES', 'AUTOMOTIVE PAINT SHOPS', 'TRANSPORTATION SERVICES  MISC',
                 'TYRE RETREADING & REPAIR', 'TAXICABS & LIMOUSINES', 'SERVICE STATIONS'],
                
                'retail':
                ['GIFT CARD NOVELTY & SOUVENIR', 'MISCELLANEOUS & SPECIALITY RET', 'WOMENS READY TO WEAR STORES',
                'FURNITURE HOME FURNISHING', 'JEWELLERY STORES WATCHES CLOCK', 'PET SHOPS PET FOODS & SUPPLIES',
                'SPORTING GOODS STORES', 'FLORISTS', 'HOUSEHOLD APPLIANCE STORES', 'MENS & WOMENS CLOTHING STORES',
                'ARTISTS SUPPLY & CRAFT SHOPS', 'MISCELLANEOUS GENERAL MERCHAND', 'GLASS PAINT & WALLPAPER STORES',
                'HOBBY TOYS & GAME SHOPS', 'ART DEALERS & GALLERIES', 'STATIONERY OFFICE/SCHOOL STORE',
                'ANTIQUE SHOPS-SALES REPAIRS RE', 'CHILDRENS & INFANTS WEAR STORE', 'CIGAR STORES & STANDS',
                'SHOE STORES', 'BAKERIES', 'MUSIC STORES INSTRUMENTS PIANO', 'MENS & BOYS CLOTHING & ACSSORS',
                'SEWING NEEDLEWORK FABRIC GOODS', 'BOOK SHOPS', 'FAMILY CLOTHING STORES', 'SPORTS APPEREL RIDING APPEREL',
                'WOMENS ACCESSORY & SPECIALITY', 'MISCELLANEOUS HOME FURNISHING', 'COSMETIC STORES', 'ELECTRONICS STORES'],
                
                'professional services':
                ['BUSINESS SERVICES MISC', 'PROFESSIONAL SERVICES', 'LEGAL SERVICES ATTORNEYS',
                'SCHOOLS & EDUCATIONAL SERVICES', 'ARCHITECTURAL ENGINEERING', 'MISCELLANEOUS PERSONAL SERVICE',
                'CHILD CARE SERVICES', 'RECREATION SERVICES', 'FUNERAL SERVICE & CREMATORIES',
                'ACCOUNTING AUDITING BOOKEEPING', 'COLLEGES UNIVERSITIES', 'MANAGEMENT CONSULTING & PUBLIC',
                'ADVERTISING SERVICES', 'TAILORS SEAMSTRESSES ALTERATNS'],
                
                'industrial':
                ['LUMBER & BUILDING MATERIALS', 'SPECIAL TRADE CONTRACTORS', 'INDUSTRIAL SUP-UNCLASS (P.CRD)',
                'NURSERIES LAWN & GARDEN SUPPLY', 'FLOOR COVERING STORES', 'PLUMBING/HEAT-EQUIP/SUP(P.CRD0',
                'DRAPERY WINDOWCOVERING & UPHOL', 'ELEC. PARTS/EQUIPMENT (P.CARD)', 'HEATING PLUMBING AIR CONDITION',
                'HARDWARE STORES', 'HRDWARE EQUIP/SUPPLIES(P.CARD)', 'GENERAL CONTRACTORS',
                'COMMERCIAL EQPT-UNCLASS(P.CRD)', 'CONSTRUCTION MAT-UNCLASS (P/C)', 'ELECTRICAL CONTRACTORS',
                'LANDSCAPING & HORTICULTURAL', 'MASONRY STONEWORK PLASTERING', 'ROOFING & SIDING SHEET METAL']}

In [17]:
MCC_Desc_list = []
industry_list = []
for key, value in industry_dict.items():
    for desc in value:
        MCC_Desc_list.append(desc)
        industry_list.append(key)

industry_df = pd.DataFrame({'MCC_Description': MCC_Desc_list, 'Industry': industry_list})

In [18]:
final_df = final_df.merge(industry_df, 'left', on = 'MCC_Description')

In [19]:
final_df['Industry'] = final_df['Industry'].fillna('other')

In [20]:
final_df = final_df[['Hashed_ID', 'land_month', 'land_month_no', 'Tenure',
       'Average_Turnover', 'Average_Trans_Count', 'Times_Marketed_To',
       'Message', 'Xer', 'mcc_rating', 'Pam_Score', 'Industry', 'responded', 'approved', 'funded']]

In [21]:
final_df.to_csv('propensity_dataset.csv', index = False)

In [22]:
to_sql_fast(final_df, name = 'pam.propensity_dataset', engine = ENGINE, if_exists = 'replace')